# UnpoolAndNormal

In [1]:
import tensorflow as tf
import numpy as np

在实现ZFNet论文中可视化(DeConv)之前,我们先来看看DeConv的流程.

反卷积网络(转置卷积网络)最早出现在该论文[Adaptive deconvolutional networks for mid and high level feature learning](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.849.3679&rep=rep1&type=pdf),是用于无监督学习的,但是在ZFNet paper中或者我们现在所说的Visualization并不具备学习功能,只是用于可视化一个已经训练完毕的模型.

其主要过程为:

UnMaxpool==> UnReLu ==> DeConv(Transpose CONV)

**UnMaxpool**

上池化(反池化):我们知道Max Pooling是不可逆的,而论文作者给出了一个近似的方式(也就是我们CNN Forward and Backward(Theory01)使用的方式)

<img src="../../../../picture/215.png" height=500 width=500>

**UnReLu**

在AlexNet中,relu函数是用于保证每层输出的激活值是正的,因此对于反向过程,我们同样是需要特征图的输出是正值,也就是说这个反激活过程和正向激活过程没有什么区别,直接应用relu函数即可.

**Transpose Convolution (Blue is input, cyan is output)**

<img src="https://cdn-images-1.medium.com/max/1600/0*CJYLcAXhmOepbMmh" height=300 width=300>

实际上deConv(反卷积)操作就是如上图所示,其也是一个卷积过程,只不过需要使用转置($rot180$)的卷积核去做卷积操作.

所以如果要实现visualization只要能够搭建以上三个步骤即可,用ZFNet中给出的过程即:

<img src="../../../../picture/214.png" height=500 width=500>

所以我们的整个可视化过程为:

<span style="color:blue">输入==>卷积==>relu==>池化==>反池化==>得到特征图==>反relu==>反卷积==>输出.</span>

比如我们要查看AlexNet中CONV5的特征图反卷积可视化,那么我们需要将图片正向流动至CONV5得到特征图,接着进行反池化,反激活,反卷积这样一个过程直到输入层($227\times 227\times 3$)即可.

那么接下去,我们先使用Tensorflow来搭建Unpool,值得注意的是,在Tensorflow 1.3 版本中并没有直接实现Unpool的函数,所以我们需要手动实现.

### UnPool

虽然在Tensorflow中没有可以直接实现Unpool的函数,但是其提供了可以记录Pool layer最大值索引的函数:

[tf.nn.max_pool_with_argmax](https://www.tensorflow.org/api_docs/python/tf/nn/max_pool_with_argmax),该函数将记录argmax index,且是在```[0,height)*[0,weights)```展平之前的索引.

![](../../../../picture/229.png)

可能这样我们还是不明白argmax返回的到底是什么,那么我们下面来举一个简单的例子:

首先我们先模拟创建一个数据集,其大小为$(2\times 2\times 2\times 3)$,Pool参数为$k=[1,2,2,1],s=[1,2,2,1],p="VALID"$

In [2]:
with tf.Session() as sess:
    original = tf.random.uniform([2,2,2,3],seed=5)
    img = sess.run(original)
    print('The Original :\n',img)
    print('First Sample R Channel First window:\n',img[0,0:2,0:2,0])
    print('Second Sample R Channel First window:\n',img[1,0:2,0:2,0])

The Original :
 [[[[0.38831127 0.6636828  0.19191074]
   [0.70605266 0.9631834  0.32628322]]

  [[0.70963013 0.6480061  0.13352966]
   [0.5734824  0.446828   0.44780207]]]


 [[[0.5382638  0.7211925  0.7389065 ]
   [0.30806708 0.11909926 0.1696955 ]]

  [[0.09324336 0.6628562  0.85271657]
   [0.69313943 0.95987093 0.49260855]]]]
First Sample R Channel First window:
 [[0.38831127 0.70605266]
 [0.70963013 0.5734824 ]]
Second Sample R Channel First window:
 [[0.5382638  0.30806708]
 [0.09324336 0.69313943]]


可以看到第一个样本和第二个样本的R通道的第一个pool窗口如上显示,现在我们使用```tf.nn.max_pool_with_argmax```

In [3]:
with tf.Session() as sess:
    original = tf.random.uniform([2,2,2,3],seed=5)
    P = tf.nn.max_pool_with_argmax(original,[1,2,2,1],[1,2,2,1],"VALID")
    pooled,ind = sess.run(P)
    print('pooled shape {},pooled: \n {}'.format(pooled.shape,pooled))
    print('ind shape {}, ind:\n {}'.format(ind.shape,ind))

pooled shape (2, 1, 1, 3),pooled: 
 [[[[0.70963013 0.9631834  0.44780207]]]


 [[[0.69313943 0.95987093 0.85271657]]]]
ind shape (2, 1, 1, 3), ind:
 [[[[ 6  4 11]]]


 [[[21 22 20]]]]


可以看到第一个样本的第一个窗口中最大值确实是$0.70963013$,第二个样本的第一个窗口的值也确实是$0.69313943$,这里没有任何问题,但是我们继续看返回的```ind```,会发现第一个样本的第一个窗口的最大值索引为$6$,发现第二个样本的第一个窗口的最大值索引为$21$,这是为什么呢?

接下去我们将原数据做一些改变:

In [4]:
with tf.Session() as sess:
    original = tf.random.uniform([2,2,2,3],seed=5)
    img = sess.run(original)
    print('The Original Reshape :\n',img.reshape(2*2*2,3))

The Original Reshape :
 [[0.38831127 0.6636828  0.19191074]
 [0.70605266 0.9631834  0.32628322]
 [0.70963013 0.6480061  0.13352966]
 [0.5734824  0.446828   0.44780207]
 [0.5382638  0.7211925  0.7389065 ]
 [0.30806708 0.11909926 0.1696955 ]
 [0.09324336 0.6628562  0.85271657]
 [0.69313943 0.95987093 0.49260855]]


我们单独来看R通道(第一列),发现$0.70963013$的索引确实是$6$(从左往右数),$0.69313943$的索引也确实是$21$(从左往右数的最后一个).

所以我们就知道,```ind```是将所有的样本全部铺平后得到的结果,那么我们在进行Unpool的时候也使用同样的方式用```ind```将```pooled```放回到最大位置即可.

由于```ind```是由所有的样本全部铺平后得到的结果,而```pooled```又和```ind```一一对应,我们可以将```ind```和```pooled```完全展平,这样就无需做for loop 按照通道一一放回,使得UnPool的运算就会快很多,我们可以先使用numpy尝试:

In [5]:
with tf.Session() as sess:
    original = tf.random.uniform([2,2,2,3],seed=5)
    P = tf.nn.max_pool_with_argmax(original,[1,2,2,1],[1,2,2,1],"VALID")
    pooled,ind = sess.run(P)

    print('ind:\n {}'.format(ind))
    ind_flatten = ind.reshape(2*1*1*3)
    print('ind flatten:\n {}'.format(ind_flatten))
    
    print('pooled: \n {}'.format(pooled))
    pooled_flatten = pooled.reshape(2*1*1*3)
    print('pooled flatten:\n {}'.format(pooled_flatten))
    
    unpool = np.zeros((2*2*2*3))
    print('unpool flatten:\n {}'.format(unpool))
    unpool[ind_flatten] = pooled_flatten
    print('unpool flatten result:\n {}'.format(unpool))
    print('unpool reshape to (2*2*2,3):\n {}'.format(unpool.reshape(2*2*2,3)))

ind:
 [[[[ 6  4 11]]]


 [[[21 22 20]]]]
ind flatten:
 [ 6  4 11 21 22 20]
pooled: 
 [[[[0.70963013 0.9631834  0.44780207]]]


 [[[0.69313943 0.95987093 0.85271657]]]]
pooled flatten:
 [0.70963013 0.9631834  0.44780207 0.69313943 0.95987093 0.85271657]
unpool flatten:
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
unpool flatten result:
 [0.         0.         0.         0.         0.9631834  0.
 0.70963013 0.         0.         0.         0.         0.44780207
 0.         0.         0.         0.         0.         0.
 0.         0.         0.85271657 0.69313943 0.95987093 0.        ]
unpool reshape to (2*2*2,3):
 [[0.         0.         0.        ]
 [0.         0.9631834  0.        ]
 [0.70963013 0.         0.        ]
 [0.         0.         0.44780207]
 [0.         0.         0.        ]
 [0.         0.         0.        ]
 [0.         0.         0.85271657]
 [0.69313943 0.95987093 0.        ]]


可以看到结果是Unpool的结果是正确的,Pool的最大值回到了最大索引的位置.最后将结果reshape到(2,2,2,3)即可完成Unpool.

既然Numpy的成功了,接下去把代码转换成Tensorflow的形式即可:

In [6]:
def _Unpool_with_with_argmax(sess, pooled, ind, out_size):
    """
    Unpool layer.
    Note:
    ======================================================================================
    In Tensorflow, the tensor can view but can't change,or say max_index-max_value change.
    So, we need it's function:tf.compat.v1.scatter_nd_update
    ==>
    https://www.tensorflow.org/api_docs/python/tf/scatter_nd_update
    Update max value to max index
    ======================================================================================

    :param pooled (tensor): pooling layer result.
    :param ind (tensor): pooling layer max value index.
    :param out_size (tensor): unpooling result.
    :return
        un_pool (tensor): unpool result.
    """
    m, h, w, c = out_size
    _, h_, w_, c_ = pooled.shape.as_list()

    ref = tf.Variable(tf.zeros([m * h * w * c]))
    pooled_ = tf.reshape(pooled, [m * h_ * w_ * c_])
    ind_ = tf.reshape(ind, [m * h_ * w_ * c_])
    sess.run(ref.initializer)

    # expand dims.
    indices = tf.expand_dims(ind_, axis=1)
    # update tensor.
    un_pool = tf.compat.v1.scatter_nd_update(ref, indices=indices, updates=pooled_)
    # reshape to (m, h, w, c)
    un_pool = tf.reshape(un_pool, (m, h, w, c))

    return un_pool

.```Unpool_with_with_argmax``` 接受三个参数:

- pooled: 池化后的结果,shape [batch_size,p_h,p_w,p_c].

- ind: 池化后感受野内最大值对应的索引值(展平) shape [batch_size,p_h,p_w,p_c].

- out_size: Unpool输出的大小.

使用Tensor的不便之处在于在Tensorflow中,tensor可以查看,但是不能直接使用索引更改,所以我们需要使用该函数:

[tf.compat.v1.scatter_nd_update](https://www.tensorflow.org/api_docs/python/tf/scatter_nd_update)

官方示例如下:

```python
ref = tf.Variable([1, 2, 3, 4, 5, 6, 7, 8])
indices = tf.constant([[4], [3], [1] ,[7]])
updates = tf.constant([9, 10, 11, 12])
update = tf.compat.v1.scatter_nd_update(ref, indices, updates)
with tf.compat.v1.Session() as sess:
     print sess.run(update)```

可以看到,```indices```是一个二维的向量,而我们将```ind```展平是一维的,所以需要扩展```axis=1```维度,即代码中的```indices = tf.expand_dims(ind_, axis=1)```

到此Unpool过程完毕,实际上只完成反卷积过程可视化出来的结果是不良好的,所以我们需要将反卷积出来的结果进行Normal.

### Normalization

那么Normalization的方式有两种:1.std,2.range.

#### 1. Standard

标准化,也是Keras的Visualization使用的方式,以下代码都比较简单就不多阐述了.

In [7]:
def _Normal_Std(self,filter_in):
    """
    Normalization of conv2d filters for visualization:
    ===========================================================================
    https://github.com/jacobgil/keras-filter-visualization/blob/master/utils.py
    ===========================================================================

    :param filter_in (ndarray): [h_size,w_size,channels]
    :return:
    """
    x = filter_in
    x -= x.mean()
    x /= (x.std() + 1e-5)
    # make most of the value between [-0.5, 0.5]
    x *= 0.1
    # move to [0, 1]
    x += 0.5
    x *= 255
    x = np.clip(x, 0, 255).astype('uint8')

    return x

#### 2. range

除了标准化,也可以使用归一化,但是可视化的色彩强度并没有标准化强.

In [8]:
def _Normal_Range(self, filter_in):
    """
    Normal ones.
    ===========================================
    New_value = (old_Value - min) / (max - min)
    ===========================================

    :param filter_in (ndarray): [h_size,w_size,channels]
    :return: normal ones filter_in.
    """
    f_min = np.amin(filter_in)
    f_max = np.amax(filter_in)

    return (filter_in - f_min) * 1.0 / (f_max - f_min + 1e-5) * 255.0

### Visualization 
---------------

下面来看看在AlexNet训练完毕后的模型应用Visualization的结果.

<table border="0">
	<tbody>
    <tr>
			<td width="10%" align="center"><h3>Layer 1</h3></td>
			<td > <img src=" visresult/Original_layer1.jpg" style="max-width:100%;"></td>
            <td > <img src=" visresult/DeConv_layer1.jpg" style="max-width:100%;"></td>
	</tr>
    <tr>
			<td width="10%" align="center"><h3>Layer 2</h3></td>
			<td > <img src=" visresult/Original_layer2.jpg" style="max-width:100%;"></td>
            <td > <img src=" visresult/DeConv_layer2.jpg" style="max-width:100%;"></td>
	</tr>
    <tr>
			<td width="10%" align="center"><h3>Layer 3</h3></td>
			<td > <img src=" visresult/Original_layer3.jpg" style="max-width:100%;"></td>
            <td > <img src=" visresult/DeConv_layer3.jpg" style="max-width:100%;"></td>
	</tr>
    <tr>
			<td width="10%" align="center"><h3>Layer 4</h3></td>
			<td > <img src=" visresult/Original_layer4.jpg" style="max-width:100%;"></td>
            <td > <img src=" visresult/DeConv_layer4.jpg" style="max-width:100%;"></td>
	</tr>
    <tr>
			<td width="10%" align="center"><h3>Layer 5</h3></td>
			<td > <img src=" visresult/Original_layer5.jpg" style="max-width:100%;"></td>
            <td > <img src=" visresult/DeConv_layer5.jpg" style="max-width:100%;"></td>
	</tr>
	</tbody>
</table>

下面我们来看看上面的可视化结果是如何产生的.